In [17]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#NumPy是一个科学计算的工具包，这里通过NumPy工具包生成模拟数据集
from numpy.random import RandomState

tf.compat.v1.disable_eager_execution()
#定义训练数据集batch的大小
batch_size = 8

#定义神经网络的参数，这里和前向传播的神经网络结构一致
w1 = tf.Variable(tf.random.normal((2,3),stddev=1,seed=1))
w2 = tf.Variable(tf.random.normal((3,1),stddev=1,seed=1))

#在shape的一个维度上使用None可以方便的使用不同的batch大小。
#在训练时需要把数据分成比较小的batch，但是在测试时，可以一次性使用全部的数据。
#当训练集比较小时这样比较方便测试，当数据集比较大时，将大量的数据放入一个batch可能会导致溢出
x=tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

#定义神经网络的前向传播过程
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#定义损失函数和反向传播的算法
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(
    y_ *tf.log(tf.clip_by_value(y,1e-10,1.0))
    +y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

#通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)
#定义规则来给出样本的标签。在这里所有x1+x2<1的样例都被认为是正样本（比如零件合格），
#而其他负样本（比如零件不合格）。和Tensorflow游乐场中表示法不太一样的是，
#在这里使用0来表示负样本，1来表示正样本。大部分解决分类问题的神经网络都会采用
#0和1的表示方法
Y=[[int(x1+x2<1)]for (x1,x2) in X]

#创建一个会话来运行Tensorflow程序
with tf.compat.v1.Session() as sess:
    init_op = tf.global_variables_initializer()
    #初始化变量
    sess.run(init_op)
    
    print('训练之前神经网络的参数的值')
    print('w1=',sess.run(w1))
    print('w2=:',sess.run(w2))

    #设定训练的次数
    STEPS = 5000
    for i in range(STEPS+1):
        #每次选取batbatch_size个样本进行训练
        start = (i * batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)

        #通过选取的样本训练神经网络并更新参数。
        sess.run(train_step,
                 feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%1000 == 0:
            #每隔一段时间计算在所有数据上的交叉熵并输出
            total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
            print('After %d training steps,cross entropy on all data is %g'%(i,total_cross_entropy))
            
            '''  
          输出结果
          After 0 training steps,cross entropy on all data is 0.219988
          After 1000 training steps,cross entropy on all data is 0.0513775
          After 2000 training steps,cross entropy on all data is 0.0249217
          After 3000 training steps,cross entropy on all data is 0.0151669
          After 4000 training steps,cross entropy on all data is 0.0101964
          交叉熵逐渐变小，交叉熵越小说明预测的结果与真实的结果差距越小
          '''  
    print('训练之后神经网络的参数的值')
    print('w1=',sess.run(w1))
    print('w2=:',sess.run(w2))


训练之前神经网络的参数的值
w1= [[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
w2=: [[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training steps,cross entropy on all data is 0.219988
After 1000 training steps,cross entropy on all data is 0.0513775
After 2000 training steps,cross entropy on all data is 0.0249217
After 3000 training steps,cross entropy on all data is 0.0151669
After 4000 training steps,cross entropy on all data is 0.0101964
After 5000 training steps,cross entropy on all data is 0.00726771
训练之后神经网络的参数的值
w1= [[-2.5009384  3.045668   2.769615 ]
 [-4.022965   1.5499796  3.2556906]]
w2=: [[-2.2671835]
 [ 3.2480507]
 [ 2.3599126]]


In [1]:
# 训练之前神经网络的参数的值
# w1= [[-0.8113182   1.4845988   0.06532937]
#  [-2.4427042   0.0992484   0.5912243 ]]
# w2=: [[-0.8113182 ]
#  [ 1.4845988 ]
#  [ 0.06532937]]
# After 0 training steps,cross entropy on all data is 0.219988
# After 1000 training steps,cross entropy on all data is 0.0513775
# After 2000 training steps,cross entropy on all data is 0.0249217
# After 3000 training steps,cross entropy on all data is 0.0151669
# After 4000 training steps,cross entropy on all data is 0.0101964
# 训练之后神经网络的参数的值
# w1= [[-2.5006778  3.045427   2.7692232]
#  [-4.022722   1.5497587  3.2552838]]
# w2=: [[-2.2669668]
#  [ 3.2477696]
#  [ 2.35954  ]]

